# ADL Identification

<br>

<br>

Les outils d'intelligence artificielle permettent de classifier automatiquement les ADL à partir de données d'accélérométrie. La qualité de l'exploration des signaux réalisée précédemment dans le notebook Clustering.ipynb reposait sur la construction de features calculés à partir des signaux brutes. Ces features calculés 'à la main' nécessitent une connaissance à priori sur la forme des signaux. 
Avec un apprentissage prodond, les réseaux de neurones sont capable de construire automatiquement des features pendant la phase d'apprentissage. Le réseau de neurones est en mesure de classifier automatiquement les signaux brutes d'accélérométrie en ADL.

Pour cette étude de classification d'ADL, on dispose de signaux d'accélérométrie du jeu de données publique [Dataset for ADL Recognition with Wrist-worn Accelerometer Data Set](https://archive.ics.uci.edu/ml/datasets/Dataset+for+ADL+Recognition+with+Wrist-worn+Accelerometer). Les données brutes sont mesurées avec un bracelet électronique porté par plusieurs individus effectuant un total de 14 ADL. Le bracelet mesure l'accélération au niveau du poignet selon les trois axes spatiaux x, y et z avec une fréquence de 25 Hz. Les données processées, qui serviront de point de départ à cette analyse, contiennent ces mêmes signaux échantillonées sur une fennêtre de 4 sec sans chevauchement.

La classification d'ADL est réalisée à partir d'un réseau de neuronne convolutif (CNN), construit à partir de l'application Keras. Une architecture relativement simple sera élaborée puis optimisée. Le notebook est composé de trois parties : 
1. Data Preparation : les données sont formatés pour être lues par un NN
2. Apprentissage : le CNN conçu est entraîné et optimisé à partir des données
3. Evaluation : les performances du modèle sont évaluées



In [36]:
import numpy as np
import pandas as pd

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import model_from_json
import tensorflow as tf

#from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio

In [30]:
# plot theme
pio.templates.default = "seaborn"

## 1. Data Préparation

On charge les données d'accélérométries échantillonnées avec une fenêtre de 4 secondes ainsi que leur label. On réalise une préparation des données pour l'entraînement de neuronnes avec :
- une conversion des jeux de données x, y, z, r en un unique tableau Numpy
- un encodage One Hot des labels
- une séparation entraînement-test 80-20 %

In [2]:
def featFile_toNp(fileprefix):
    '''
    load the 4 datasets x, y, z, r into numpy array and 
    combine them into 4-dimensional array
    
    ex fileprefix :   '../ADL_class/mydata/data_feature_nn_'
    of file full name '../ADL_class/mydata/data_feature_nn_x.csv'
    '''
    
    dfout = []
    
    directions = ['x', 'y', 'z', 'r']
    
    
    # loop over directions x, y ...
    for X in directions:
        
        # add X dataframe to df
        df_tmp = np.genfromtxt(fileprefix + X + '.csv',delimiter=',')
        
        dfout.append(df_tmp)
        
    # stack all direction df into 1 df
    dfout = np.dstack(dfout)
    
    return dfout

In [3]:
df = featFile_toNp('../ADL_class/mydata/data_feature_nn_')
print('Input data dimension {}'.format(df.shape))

Input data dimension (3090, 100, 4)


*Les signaux échantillonés proviennent de 4 tables pour les directions x, y, z et r. Elles sont fusionnées dans un tableau Numpy.*

In [4]:
# load ADL labels
my = pd.read_csv('../ADL_class/mydata/y_nn.csv', header = None)

# convert labels into pandas Category
my = pd.Categorical(my[0])

# dictionnary for mappring index -> ADL (0 : 'Brush_teeth')
label_id = {}
for index, val in enumerate(my.categories):
    label_id[index] = val

# Label One Hot Encoded
myc = to_categorical(my.codes)

*Les labels sont chargés et convertis en données numériques à partir d'un encodage One Hot.*

In [5]:
# train test split
mx_tr, mx_te, my_tr, my_te = train_test_split(df, myc, train_size = 0.80)

print('Training data input shape : {}'.format(mx_tr.shape))
print('Testing data input shape : {}'.format(mx_te.shape))

Training data input shape : (2472, 100, 4)
Testing data input shape : (618, 100, 4)


*Les données d'entrée et labels sont séparés en un jeu d'entraînement et un jeu de test dans des proportions 80 et 20 % respectivement.*

## 2. Apprentissage

Le réseau de neuronne est construit pour réaliser une tâche de classification multi-class avec la fonction de coût categorical cross entropy. On défini un modèle avec deux couches convolutives unidimensionnelles, suivies par une couche *dropout* pour régularisation, puis une couche de *pooling*. À la suite du *pooling*, les features appris sont *flattened* en un long vecteur et passés à une couche complètement connectée avant une couche de sortie utilisée pour la prédiction finale.

We will define the model as having two 1D CNN layers, followed by a dropout layer for regularization, then a pooling layer. After pooling, the learned features are flattened to one long vector and pass through a fully connected layer before the output layer used to make a prediction. 

Le réseau de neurone est entraîné/évalué avec une validation croisée en 3 strates. Plusieurs configurations d'hyperparamètres sont testées avec une recherche sur grille. Les hyperparamètres réglés sont les suivants :
- learning rate : [0.001, 0.0001]
- kernel size : [10, 20, 30]
- batch size : [32, 128]
- number of epochs : [15, 30]

Les accuracy des modèles de la recherche sur grille sont affichées sur le diagramme de dispersion ci-dessous, en fonction de l'erreur sur l'accuracy (déviation standard sur validation croisée). On observe une relativement grande dispersion des modèles en terme d'accuracy, soulignant l'importance de la recherche sur grille pour réaliser la sélection du modèle de classification. Le modèle choisi est celui qui maximise l'accuracy, avec un temps de calcul moyen de 30 secondes (XX). Le meilleur modèle permet d'atteindre une accuracy moyenne de 70 % sur validation croisée. Les meilleurs hyperparamètres sont les suivant : `learning_rate`: 0.001,`kernel_size`: 10, `epochs`: 30, `batch_size`: 32.

In [6]:
# Creates a model given an activation and learning rate
def create_model(kernel_size,
                 learning_rate = 0.01, 
                 activation = 'relu',
                 #n_timesteps = 128,
                 #n_features = 3,
                 #n_outputs = 6

                ):
    
    n_timesteps = 100  # 128   
    n_features =  4    #   3   
    n_outputs =   14   #  6   
    filters= 64 
    # Create an Adam optimizer with the given learning rate
    opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
    
    
    # Create your binary classification model  
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation=activation))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation=activation))
    model.add(Dense(n_outputs, activation='softmax'))
    
    # Compile your model with your optimizer, loss, and metrics
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [7]:
# Create a KerasClassifier
model = KerasClassifier(build_fn = create_model,
                        #epochs= 20, batch_size = 128
                       )

# Define the parameters to try out
params = {#'activation': ['relu'],
          'kernel_size' : [10, 20, 30],
          'batch_size': [32, 128], 
          'epochs': [15, 30],
          'learning_rate': [0.001, 0.0001]
}

# Create a randomize search cv object passing in the parameters to try
random_search = RandomizedSearchCV(model, param_distributions = params, cv = KFold(3))

# Running random_search.fit(X,y) would start the search,but it takes too long!
random_search.fit(mx_tr, my_tr)

Epoch 1/30
13/13 [==============================] - 2s 91ms/step - loss: 2.5662 - accuracy: 0.1729
Epoch 2/30
13/13 [==============================] - 1s 105ms/step - loss: 2.2142 - accuracy: 0.2816
Epoch 3/30
13/13 [==============================] - 1s 101ms/step - loss: 1.9238 - accuracy: 0.4029
Epoch 4/30
13/13 [==============================] - 1s 91ms/step - loss: 1.7352 - accuracy: 0.4290
Epoch 5/30
13/13 [==============================] - 1s 85ms/step - loss: 1.6408 - accuracy: 0.4478
Epoch 6/30
13/13 [==============================] - 1s 86ms/step - loss: 1.5776 - accuracy: 0.4527
Epoch 7/30
13/13 [==============================] - 1s 101ms/step - loss: 1.5181 - accuracy: 0.4691
Epoch 8/30
13/13 [==============================] - 1s 98ms/step - loss: 1.4948 - accuracy: 0.4909
Epoch 9/30
13/13 [==============================] - 1s 90ms/step - loss: 1.4716 - accuracy: 0.4921
Epoch 10/30
13/13 [==============================] - 1s 107ms/step - loss: 1.4323 - accuracy: 0.5225
Epoch

13/13 [==============================] - 2s 142ms/step - loss: 1.2180 - accuracy: 0.5874
Epoch 21/30
13/13 [==============================] - 1s 117ms/step - loss: 1.2006 - accuracy: 0.5947
Epoch 22/30
13/13 [==============================] - 2s 122ms/step - loss: 1.1658 - accuracy: 0.5941
Epoch 23/30
13/13 [==============================] - 2s 123ms/step - loss: 1.1508 - accuracy: 0.5989
Epoch 24/30
13/13 [==============================] - 2s 122ms/step - loss: 1.1325 - accuracy: 0.6183
Epoch 25/30
13/13 [==============================] - 2s 121ms/step - loss: 1.1095 - accuracy: 0.6153
Epoch 26/30
13/13 [==============================] - 2s 123ms/step - loss: 1.1047 - accuracy: 0.6159
Epoch 27/30
13/13 [==============================] - 2s 134ms/step - loss: 1.0812 - accuracy: 0.6280
Epoch 28/30
13/13 [==============================] - 1s 114ms/step - loss: 1.0700 - accuracy: 0.6347
Epoch 29/30
13/13 [==============================] - 2s 127ms/step - loss: 1.0451 - accuracy: 0.6371
Ep

13/13 [==============================] - 1s 85ms/step - loss: 1.4429 - accuracy: 0.5121
Epoch 10/30
13/13 [==============================] - 1s 83ms/step - loss: 1.4177 - accuracy: 0.5376
Epoch 11/30
13/13 [==============================] - 1s 100ms/step - loss: 1.3772 - accuracy: 0.5504
Epoch 12/30
13/13 [==============================] - 1s 84ms/step - loss: 1.3601 - accuracy: 0.5516
Epoch 13/30
13/13 [==============================] - 1s 97ms/step - loss: 1.3415 - accuracy: 0.5558
Epoch 14/30
13/13 [==============================] - 1s 95ms/step - loss: 1.3001 - accuracy: 0.5740
Epoch 15/30
13/13 [==============================] - 1s 96ms/step - loss: 1.3065 - accuracy: 0.5661
Epoch 16/30
13/13 [==============================] - 1s 92ms/step - loss: 1.2764 - accuracy: 0.5819
Epoch 17/30
13/13 [==============================] - 1s 75ms/step - loss: 1.2613 - accuracy: 0.5710
Epoch 18/30
13/13 [==============================] - 1s 82ms/step - loss: 1.2510 - accuracy: 0.5959
Epoch 19/30

52/52 [==============================] - 1s 23ms/step - loss: 0.4245 - accuracy: 0.8562
Epoch 30/30
26/26 [==============================] - 0s 5ms/step - loss: 1.0337 - accuracy: 0.6881
Epoch 1/30
52/52 [==============================] - 1s 21ms/step - loss: 1.8493 - accuracy: 0.3780
Epoch 2/30
52/52 [==============================] - 1s 21ms/step - loss: 1.5051 - accuracy: 0.4897
Epoch 3/30
52/52 [==============================] - 1s 21ms/step - loss: 1.4066 - accuracy: 0.5407
Epoch 4/30
52/52 [==============================] - 1s 21ms/step - loss: 1.2870 - accuracy: 0.5680
Epoch 5/30
52/52 [==============================] - 1s 21ms/step - loss: 1.1672 - accuracy: 0.5934
Epoch 6/30
52/52 [==============================] - 1s 21ms/step - loss: 1.0905 - accuracy: 0.6177
Epoch 7/30
52/52 [==============================] - 1s 22ms/step - loss: 1.0628 - accuracy: 0.6250
Epoch 8/30
52/52 [==============================] - 1s 21ms/step - loss: 1.0144 - accuracy: 0.6614
Epoch 9/30
52/52 [===

13/13 [==============================] - 1s 81ms/step - loss: 2.1217 - accuracy: 0.3434
Epoch 3/15
13/13 [==============================] - 1s 70ms/step - loss: 1.8668 - accuracy: 0.4144
Epoch 4/15
13/13 [==============================] - 1s 64ms/step - loss: 1.6997 - accuracy: 0.4527
Epoch 5/15
13/13 [==============================] - 1s 72ms/step - loss: 1.5908 - accuracy: 0.4848
Epoch 6/15
13/13 [==============================] - 1s 63ms/step - loss: 1.5203 - accuracy: 0.4951
Epoch 7/15
13/13 [==============================] - 1s 69ms/step - loss: 1.4653 - accuracy: 0.5152
Epoch 8/15
13/13 [==============================] - 1s 63ms/step - loss: 1.4351 - accuracy: 0.5176
Epoch 9/15
13/13 [==============================] - 1s 64ms/step - loss: 1.4007 - accuracy: 0.5237
Epoch 10/15
13/13 [==============================] - 1s 65ms/step - loss: 1.3747 - accuracy: 0.5449
Epoch 11/15
13/13 [==============================] - 1s 71ms/step - loss: 1.3479 - accuracy: 0.5607
Epoch 12/15
13/13 [

13/13 [==============================] - 1s 92ms/step - loss: 1.6546 - accuracy: 0.4545
Epoch 6/15
13/13 [==============================] - 1s 87ms/step - loss: 1.5891 - accuracy: 0.4660
Epoch 7/15
13/13 [==============================] - 1s 80ms/step - loss: 1.5437 - accuracy: 0.4830
Epoch 8/15
13/13 [==============================] - 1s 86ms/step - loss: 1.4922 - accuracy: 0.4915
Epoch 9/15
13/13 [==============================] - 1s 80ms/step - loss: 1.4526 - accuracy: 0.5067
Epoch 10/15
13/13 [==============================] - 1s 79ms/step - loss: 1.4370 - accuracy: 0.5279
Epoch 11/15
13/13 [==============================] - 1s 93ms/step - loss: 1.4102 - accuracy: 0.5225
Epoch 12/15
13/13 [==============================] - 1s 80ms/step - loss: 1.3739 - accuracy: 0.5352
Epoch 13/15
13/13 [==============================] - 1s 86ms/step - loss: 1.3566 - accuracy: 0.5407
Epoch 14/15
13/13 [==============================] - 1s 79ms/step - loss: 1.3279 - accuracy: 0.5607
Epoch 15/15
7/7 

52/52 [==============================] - 1s 19ms/step - loss: 0.7821 - accuracy: 0.7312
Epoch 10/30
52/52 [==============================] - 1s 19ms/step - loss: 0.7009 - accuracy: 0.7591
Epoch 11/30
52/52 [==============================] - 1s 19ms/step - loss: 0.7320 - accuracy: 0.7354
Epoch 12/30
52/52 [==============================] - 1s 19ms/step - loss: 0.7105 - accuracy: 0.7591
Epoch 13/30
52/52 [==============================] - 1s 19ms/step - loss: 0.5940 - accuracy: 0.7846
Epoch 14/30
52/52 [==============================] - 1s 18ms/step - loss: 0.5506 - accuracy: 0.8113
Epoch 15/30
52/52 [==============================] - 1s 19ms/step - loss: 0.5579 - accuracy: 0.8070
Epoch 16/30
52/52 [==============================] - 1s 19ms/step - loss: 0.5177 - accuracy: 0.8149
Epoch 17/30
52/52 [==============================] - 1s 19ms/step - loss: 0.5325 - accuracy: 0.8095
Epoch 18/30
52/52 [==============================] - 1s 19ms/step - loss: 0.4340 - accuracy: 0.8453
Epoch 19/30


13/13 [==============================] - 1s 61ms/step - loss: 0.3977 - accuracy: 0.8610
Epoch 30/30
7/7 [==============================] - 0s 13ms/step - loss: 0.9389 - accuracy: 0.6735
Epoch 1/30
13/13 [==============================] - 1s 64ms/step - loss: 1.9742 - accuracy: 0.3477
Epoch 2/30
13/13 [==============================] - 1s 65ms/step - loss: 1.5159 - accuracy: 0.4958
Epoch 3/30
13/13 [==============================] - 1s 65ms/step - loss: 1.3433 - accuracy: 0.5540
Epoch 4/30
13/13 [==============================] - 1s 64ms/step - loss: 1.2105 - accuracy: 0.6001
Epoch 5/30
13/13 [==============================] - 1s 71ms/step - loss: 1.1735 - accuracy: 0.6056
Epoch 6/30
13/13 [==============================] - 1s 64ms/step - loss: 1.1050 - accuracy: 0.6256
Epoch 7/30
13/13 [==============================] - 1s 65ms/step - loss: 1.0130 - accuracy: 0.6632
Epoch 8/30
13/13 [==============================] - 1s 64ms/step - loss: 0.9620 - accuracy: 0.6687
Epoch 9/30
13/13 [====

13/13 [==============================] - 1s 79ms/step - loss: 2.2606 - accuracy: 0.3380
Epoch 3/15
13/13 [==============================] - 1s 71ms/step - loss: 2.0051 - accuracy: 0.3774
Epoch 4/15
13/13 [==============================] - 1s 73ms/step - loss: 1.8217 - accuracy: 0.4114
Epoch 5/15
13/13 [==============================] - 1s 81ms/step - loss: 1.7276 - accuracy: 0.4411
Epoch 6/15
13/13 [==============================] - 1s 78ms/step - loss: 1.6482 - accuracy: 0.4502
Epoch 7/15
13/13 [==============================] - 1s 71ms/step - loss: 1.5945 - accuracy: 0.4545
Epoch 8/15
13/13 [==============================] - 1s 79ms/step - loss: 1.5715 - accuracy: 0.4703
Epoch 9/15
13/13 [==============================] - 1s 81ms/step - loss: 1.5279 - accuracy: 0.4745
Epoch 10/15
13/13 [==============================] - 1s 78ms/step - loss: 1.4884 - accuracy: 0.5036
Epoch 11/15
13/13 [==============================] - 1s 81ms/step - loss: 1.4712 - accuracy: 0.5097
Epoch 12/15
13/13 [

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd31c12afa0>,
                   param_distributions={'batch_size': [32, 128],
                                        'epochs': [15, 30],
                                        'kernel_size': [10, 20, 30],
                                        'learning_rate': [0.001, 0.0001]})

In [55]:
random_search.best_params_

{'learning_rate': 0.001, 'kernel_size': 10, 'epochs': 30, 'batch_size': 32}

*Jeu d'hyperparamètes qui maximise l'accuracy du modèle.*

In [8]:
dfres = pd.DataFrame(random_search.cv_results_)
dfres.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_kernel_size,param_epochs,param_batch_size,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
6,29.616349,0.354802,0.283639,0.021135,0.001,10,30,32,"{'learning_rate': 0.001, 'kernel_size': 10, 'e...",0.733010,0.692961,0.684466,0.703479,0.021167,1
7,25.875814,0.194913,0.242532,0.004045,0.001,10,30,128,"{'learning_rate': 0.001, 'kernel_size': 10, 'e...",0.719660,0.673544,0.711165,0.701456,0.020040,2
8,15.140156,0.072732,0.282959,0.024500,0.001,10,15,32,"{'learning_rate': 0.001, 'kernel_size': 10, 'e...",0.697816,0.661408,0.705097,0.688107,0.019112,3
2,36.845238,2.520910,0.307848,0.018491,0.001,20,30,32,"{'learning_rate': 0.001, 'kernel_size': 20, 'e...",0.674757,0.688107,0.688107,0.683657,0.006293,4
3,14.843135,0.298883,0.272847,0.004504,0.001,30,15,128,"{'learning_rate': 0.001, 'kernel_size': 30, 'e...",0.634709,0.609223,0.663835,0.635922,0.022312,5


*Résultats des 5 premiers modèles par ordre d'accuracy.*

In [34]:
fig1 = px.scatter(dfres, x = 'mean_test_score', y = 'std_test_score',
                 hover_data=["param_epochs", "param_batch_size"])
fig1.update_layout(xaxis_title_text = 'mean CV accuracy',
                  yaxis_title_text = 'std CV accuracy', 
                  width = 500, height = 400)

*Diagramme de dispersion des modèles en fonction de leur accuracy moyenne (x) sur validation croisée et de leur déviation standard d'accuracy (y).*

## 3. Evaluation

On retient par la suite le modèle avec la meilleure accuracy. Le modèle obtient une accuracy de 75 % (XX) sur le jeu de test labelisé. La matrice de confusion de la classification correspondante est affichée ci-dessous. Les éléments diagonaux de la matrice correpondent aux échantillons correctement classifiés par le modèle, c'est-à-dire pour lesquels la prédiction du modèle (axe y) est égale à la vérité (axe x). 

La fraction d'erreur par classe prédite est donné par la précision. On peut regrouper les ADL en trois groupes selon la précision : 
- faible (~ [50-60] %) : Getup_bed, Liedown_bed, Climb_stairs
- moyenne (~ [70 - 80] %) : Descend_stairs, Drink_glass ,Eat_meat, Eat_soup, Pour_water, Sitdown_chair, Standup_chair, Use_telephone, Walk 
- élevée (~ 90 %) : Brush_teeth, Comb_hair


Interprétation des classes prédites de faible précisions : les erreurs de prédiction de Climb_stairs viennent d'ADL Walk, comme on peut le voir sur la matrice de confusion. Cette erreur est due à la similarité des signaux de Climb_stairs et de Walk. Les erreurs sur la prédiction Getup_bed et Liedown_bed sont réparties sur plusieurs ADL, ce qui indique la difficulté du modèle à repérer le signal de ces deux ADL.

La table ci-dessous affiche la précision, recall et f1-score de la classification pour chaque ADL. Les 5 ADL les mieux classés (rangés par f1-score) sont : Brush_teeth, Comb_hair, Eat_meat, Descend_stairs et Eat_soup.

In [10]:
# retrieve best model from hyperparameter grid search
bestnn = random_search.best_estimator_

# CNN predictions on test set ADL
preds = bestnn.predict(mx_te)

# return ADL names
preds = [label_id[elt] for elt in preds]

# test set ADL true value
my_tex = [label_id[np.argmax(y, axis=None, out=None)] for y in my_te]

# compute accuracy score
acc = round(metrics.accuracy_score(my_tex, preds), 2)
print('Test dataset model accuracy : {}'.format(acc))

Test dataset model accuracy : 0.75


In [31]:
# confusion matrix : true versus predicted labels
#w2v_X = pd.crosstab(my_tex, preds)
w2v_X = pd.crosstab(pd.Series(my_tex), pd.Series(preds))

fig2  = ff.create_annotated_heatmap(w2v_X.values,#.round(2).values,
                                   x = w2v_X.columns.to_list(),
                                   y = w2v_X.index.to_list(),
                                   colorscale='Viridis')

fig2.update_layout(xaxis_title_text = 'Prediction',
                  yaxis_title_text = 'True',
                  width=700,
                  title='ADL'
                  )
fig2.update_traces(xgap=1, ygap=1)
fig2.update_xaxes(showgrid=False)
fig2.update_yaxes(showgrid=False)

*Matrice de confusion de la classification des ADL par le CNN optimal.*

In [12]:
print(metrics.classification_report(my_tex, preds))

                precision    recall  f1-score   support

   Brush_teeth       0.92      0.92      0.92        38
  Climb_stairs       0.63      0.70      0.67        57
     Comb_hair       0.91      0.86      0.89        36
Descend_stairs       0.82      0.86      0.84        21
   Drink_glass       0.77      0.79      0.78        61
      Eat_meat       0.80      0.94      0.87        48
      Eat_soup       0.89      0.80      0.84        10
     Getup_bed       0.49      0.60      0.54        57
   Liedown_bed       0.50      0.35      0.41        17
    Pour_water       0.72      0.83      0.77        52
 Sitdown_chair       0.82      0.77      0.79        30
 Standup_chair       0.70      0.66      0.68        35
 Use_telephone       0.75      0.58      0.65        26
          Walk       0.87      0.75      0.80       130

      accuracy                           0.75       618
     macro avg       0.76      0.74      0.75       618
  weighted avg       0.76      0.75      0.76 

*Précision, Recall et f1-score : 3 métriques résumant la qualité de la classification par ADL.*

In [35]:
# attempt to save and load trained CNN

## serialize model to JSON
#model_json = bestnn.model.to_json()
#with open("mydata/model/model.json", "w") as json_file:
#    json_file.write(model_json)
## serialize weights to HDF5
#bestnn.model.save_weights("mydata/model/model.h5")
#print("Saved model to disk")

## load json and create model
#json_file = open('mydata/model/model.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
## load weights into new model
#loaded_model.load_weights("mydata/model/model.h5")
#print("Loaded model from disk")

## 4. Enregistrements Non Labelisés

Enfin, le modèle est évalué sur une deux long enregistrements, le premier de 7 heures et le second de 25 heures. Le but est d'identifier la réalisation d'ADL cohérents au cours de l'enregistrement.

In [17]:
def df_prediction(in_np, model, label_id):
    '''
    return df with predict and predict_proba
    for each sample in in_np dataframe
    also add some relevant info
    '''
      
    
    # predicted proba for each class
    dfout = model.predict_proba(in_np)

    #convert numpy to pandas
    dfout = pd.DataFrame(dfout)

    # rename column label index to label name
    dfout = dfout.rename(columns=label_id)

    
    # best model prediction of input data
    best_pred = model.predict(in_np)
    
    # convert prediction into label names
    best_pred = [label_id[elt] for elt in best_pred]

    # add best prediction to final df
    dfout['best_pred'] = best_pred
    
    # add threshold informations 
    proba_threshold = [0.5, 0.7, 0.9]
    
    for elt in proba_threshold:
     
        # count number of class with a proba > elt (ex : 90 %)
        mseries = (dfout[label_id.values()] > elt).sum(axis=1)
        
        # add the flag as a column in final datafram
        colname = 'prob_thres_' + str(elt)
        dfout[colname] = mseries
    
    
    # add time info : convert sample nb in time (hours)
    dfout = dfout.reset_index()
    dfout['index']= dfout['index']  / 15 / 60
    
    
    return dfout

### Enregistrement de 7 heures

L'enregistrement de 7 heures est principalement identifier comme Getup_bed (~80%). La liste du pourcentage de temps aloué à chaque ADL est affiché sur la liste de pourcentage ci-dessous. Les autres activités significatives réalisées pendant cet enregistrement sont Brush_teeth, Use_telephone, Sitdown_chair, Standup_chair.

L'activité Getup_bed dure habituellement quelques secondes et ne s'effectue pas plus de quelques fois dans une journée, il est donc fort peut probable que les prédiction de Getup_bed soit majoritairement justes dans cette situation. Les mouvements classés comme Getup_bed peuvent provenir de mouvement semblables à l'ADL Getup_bed, ou tout simplement être trop différent des 14 ADL utilisés pour la classification. La mauvais précision de Getup_bed (49 %) permettrait d'expliquer ce dernier cas.

La distribution des échantillons est affichée ci-dessous en fonction de la probabilité de la classe Getup_bed assignée par le modèle. On remarque que les échantillons effectivement prédits Getup_bed on une assez haute probabilité Getup_bed. Ceci indique que les mouvements prédit Getup_bed sont assez semblable mouvement de Getup_bed, comme des mouvements d'inactivité.

L'évolution des classe prédites par le modèle en fonction du temps est affichée sur le diagramme de dispersion ci-dessous. On peut voir les échantillons prédits Getup_bed se répartissent sur tout la durée de l'enregistrement, renforçant l'hypothèse d'un bruit de classification. On remarque des patterns de durées compatibles avec certaines activités, comme des séquences de plusieurs échantillons successifs Comb_hair, Use_telephone ou Sitdown_chair. 
On peut aussi remarquer que l'individu va manger à 1 heure de l'entregistrement, avec l'alignement des classes prédites : Walk, Climb_stairs, Drink_glass, Eat_meat.
L'activité Brush_teeth est quand à elle beaucoup trop présente pour être réaliste.

On pourrait faire l'hypothèse que cette enregistrement mesure l'activité d'une personne travaillant au bureau devant son ordinateur, assis sur sa chaise (Sitdown_chaire), tape sur son clavier d'ordinateur (Brush_teeth), appelle des collègues (Use_telephone), et va manger 1 heure après le début de l'enregistrement. 

In [18]:
# load 4 csv files x,y,z,r into a single numpy array
long2 = featFile_toNp('mydata/NN/test2_data_feature_nn_')

# get model predictions into a datafram
dfpred_lg2 = df_prediction(long2, bestnn, label_id)
dfpred_lg2

,index,Brush_teeth,Climb_stairs,Comb_hair,Descend_stairs,Drink_glass,Eat_meat,Eat_soup,Getup_bed,Liedown_bed,Pour_water,Sitdown_chair,Standup_chair,Use_telephone,Walk,best_pred,prob_thres_0.5,prob_thres_0.7,prob_thres_0.9
0,0.000000,4.292364e-02,5.721486e-05,7.157286e-02,7.289245e-14,9.625721e-09,1.156974e-15,8.203675e-16,0.864318,5.712387e-03,1.599063e-07,1.783434e-07,7.748566e-10,1.541515e-02,7.182433e-12,Getup_bed,1,1,0
1,0.001111,2.179956e-06,2.079683e-09,4.984239e-09,7.761488e-18,5.043227e-15,1.456707e-24,1.085716e-25,0.999749,3.589317e-08,5.667676e-16,5.584713e-13,6.258773e-16,2.492401e-04,1.679066e-13,Getup_bed,1,1,1
2,0.002222,4.708079e-08,2.146245e-11,6.736126e-13,3.284918e-24,3.610124e-28,1.847859e-32,1.868662e-35,1.000000,4.490494e-11,1.526638e-25,4.141408e-15,2.197854e-19,8.544295e-11,1.007160e-20,Getup_bed,1,1,1
3,0.003333,5.318927e-01,1.719408e-02,2.126730e-01,6.621717e-10,1.081972e-05,2.242873e-12,8.015299e-15,0.200846,2.692421e-02,2.193758e-07,5.618495e-07,2.067185e-09,1.045443e-02,3.839208e-06,Brush_teeth,1,0,0
4,0.004444,1.382441e-04,1.427496e-05,1.352969e-03,5.392772e-13,5.181269e-11,1.889188e-21,2.932077e-21,0.998357,6.671709e-05,8.625114e-15,1.106572e-08,6.381742e-11,7.058533e-05,2.184054e-12,Getup_bed,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6460,7.177778,1.456677e-11,6.927287e-27,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,4.610613e-14,1.609846e-31,1.905915e-08,9.547642e-15,1.299475e-27,2.757891e-21,Getup_bed,1,1,1
6461,7.178889,8.974400e-11,1.894942e-27,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.450316e-13,1.707585e-30,2.267805e-09,4.629277e-14,1.144868e-27,2.173024e-22,Getup_bed,1,1,1
6462,7.180000,4.023242e-11,9.919025e-26,4.209318e-32,3.680736e-36,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,4.911762e-15,8.939267e-28,3.993957e-12,1.192635e-17,2.620807e-22,1.754136e-24,Getup_bed,1,1,1
6463,7.181111,5.461589e-09,8.346333e-15,3.649191e-16,4.976313e-25,2.662804e-26,1.072584e-33,1.631161e-38,1.000000,5.512598e-13,5.396668e-26,9.809005e-18,9.876288e-18,1.277609e-13,3.045893e-19,Getup_bed,1,1,1


In [56]:
dfpred_lg2['best_pred'].value_counts(normalize=True) * 100

Getup_bed        81.206497
Brush_teeth       9.280742
Use_telephone     3.511214
Sitdown_chair     3.201856
Standup_chair     1.423047
Walk              0.680588
Comb_hair         0.247486
Climb_stairs      0.139211
Pour_water        0.123743
Drink_glass       0.061872
Eat_soup          0.046404
Liedown_bed       0.046404
Eat_meat          0.030936
Name: best_pred, dtype: float64

*Pourcentage d'échantillons de l'enregistrement de 7 heures attribué à un ADL donné.*

In [20]:
# add a flag if pred = Getup_bed or not
pred_getupbed = dfpred_lg2['best_pred'] == 'Getup_bed'

dfpred_lg2['Getup_bed_flag'] = pred_getupbed 

In [38]:
fig3 = px.histogram(dfpred_lg2, x = 'Getup_bed', color='Getup_bed_flag')
fig3.update_layout(yaxis_title_text = 'Nb of samples',
                 xaxis_title_text = 'Getup_bed probability',
                  width = 500, height = 400)

In [45]:
# select predictions with > 90 % confidence 
mask09 = dfpred_lg2["prob_thres_0.9"] == 1
mask09.value_counts(normalize = True)

True     0.84826
False    0.15174
Name: prob_thres_0.9, dtype: float64

*85 % des meilleurs predictions du modèle on une probabilité de confiance supérieure à 90 %.*

In [51]:
fig4 = px.scatter(dfpred_lg2[mask09], x = 'index', y = 'best_pred')
fig4.update_layout(yaxis_title_text = 'Best prediction (> 90 % conf.)',
                  xaxis_title_text = 'time (hour)', title = 'Record #2')
fig4.update_traces(marker_size=5)

*Labels prédits par fenêtre de 4 seconde au cours de l'enregistrement #2 d'environ 7 heures.*

### Enregistrement 25 heures

Comme pour l'enregistrement de 7 heures, l'échantillon de 25 heures est dominés par des échantillons prédit Getup_bed, très probablement du bruit de classification. Après Getup_be, les 3 principales classes prédites sont Brush_teeth, Sitdown_chair et Use_telephone. 

On peut déduire à partir du peu d'échantillon Walk que l'individu reste principalement à la même place, probablement chez lui. On remarque également une plus forte inactivité de 3 à 13 heures après le début de l'enregistrement. Cette inactivité ce remarque par la plus forte proportions d'échantillon Getup_bed. Il est fort probable que l'individu soit en train de dormir durant cette période ou ait retiré son bracelet.

In [24]:
# load 4 csv files x,y,z,r into a single numpy array
long1 = featFile_toNp('mydata/NN/test1_data_feature_nn_')

# get model predictions into a datafram
dfpred_lg1 = df_prediction(long1, bestnn, label_id)
dfpred_lg1

,index,Brush_teeth,Climb_stairs,Comb_hair,Descend_stairs,Drink_glass,Eat_meat,Eat_soup,Getup_bed,Liedown_bed,Pour_water,Sitdown_chair,Standup_chair,Use_telephone,Walk,best_pred,prob_thres_0.5,prob_thres_0.7,prob_thres_0.9
0,0.000000,9.594327e-17,1.850061e-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,3.612071e-10,7.428459e-28,2.197992e-07,1.109597e-14,7.735421e-31,4.229850e-27,Getup_bed,1,1,1
1,0.001111,6.480349e-07,3.015350e-18,1.546531e-25,6.286521e-25,1.154809e-25,3.379815e-26,7.001826e-29,0.551343,7.551093e-07,9.602485e-11,4.486552e-01,4.025210e-07,4.659209e-13,1.109411e-12,Getup_bed,1,0,0
2,0.002222,6.141860e-02,3.653675e-02,2.292064e-01,5.166396e-08,6.690965e-05,4.804324e-09,8.106215e-08,0.577993,2.696109e-03,3.121936e-02,2.728461e-05,1.026721e-03,5.979811e-02,1.028678e-05,Getup_bed,1,0,0
3,0.003333,8.100113e-09,1.628291e-12,1.868969e-11,1.772986e-24,9.401274e-21,2.423809e-32,2.841632e-33,0.999958,1.439408e-13,3.827140e-21,7.193405e-20,1.722148e-20,4.218312e-05,7.890741e-20,Getup_bed,1,1,1
4,0.004444,8.640227e-12,1.303409e-09,7.566671e-13,1.049423e-16,8.700838e-27,5.583549e-25,3.410217e-30,1.000000,3.464779e-12,4.725438e-21,1.309539e-13,2.124593e-11,3.108143e-15,8.074652e-15,Getup_bed,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22660,25.177778,4.624787e-13,1.998304e-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,2.201366e-14,9.997701e-33,5.151838e-10,2.087392e-16,4.596027e-30,7.015956e-26,Getup_bed,1,1,1
22661,25.178889,5.822220e-13,2.407750e-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,3.673357e-14,1.622539e-32,6.562610e-10,3.088510e-16,7.170193e-30,1.161777e-25,Getup_bed,1,1,1
22662,25.180000,1.280584e-12,5.645366e-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,4.958443e-14,3.340073e-32,1.026279e-09,6.998910e-16,1.411750e-29,3.710177e-25,Getup_bed,1,1,1
22663,25.181111,1.479068e-12,6.909568e-30,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,5.070369e-14,4.002182e-32,1.187094e-09,8.509063e-16,1.539542e-29,4.794243e-25,Getup_bed,1,1,1


In [25]:
dfpred_lg1['best_pred'].value_counts(normalize=True)

Getup_bed         0.877344
Brush_teeth       0.059740
Sitdown_chair     0.021531
Use_telephone     0.021531
Standup_chair     0.005824
Drink_glass       0.005383
Pour_water        0.003883
Comb_hair         0.001368
Liedown_bed       0.001147
Walk              0.001103
Eat_soup          0.000529
Climb_stairs      0.000353
Eat_meat          0.000132
Descend_stairs    0.000132
Name: best_pred, dtype: float64

In [53]:
# select predictions with > 90 % confidence 
mask09 = dfpred_lg1["prob_thres_0.9"] == 1
mask09.value_counts(normalize = True)

True     0.898213
False    0.101787
Name: prob_thres_0.9, dtype: float64

In [54]:
fig5 = px.scatter(dfpred_lg1[mask09], x = 'index', y = 'best_pred')
fig5.update_layout(yaxis_title_text = 'Best prediction (> 90 % conf.)',
                  xaxis_title_text = 'time (hour)', title = 'Record #1')
fig5.update_traces(marker_size=5)

*Labels prédits par fenêtre de 4 seconde au cours de l'enregistrement #1 d'environ 25 heures.*


## Conclusion

À partir de données d'acélérométries mesurées pendant la réalisation d'ADL issues de la banque UCI, nous avons conçu et entraîné un réseau de neurones convolutif capable d'identifier 14 ADL. Il est formé d'une architecture relativement simple en 7 couches, dont deux convolutives. Le modèle optimal obtenu, à la suite d'une recherche sur grille d'hyperparamètre, atteint une accuracy de 75 % sur jeu de test.

Le modèle est évalué sur deux enregistrements non labelisés de plusieurs heures. Ses prédictions permettent de dégager des activités probables effectuées pendant les enregistrements, notamment sortir pour aller manger, dormir ou travailler. Néanmoins le modèle ne nous permet pas de suivre l'activité minute par minute de l'individu avec précision.